# 111 LangGraph: Human-in-the-Loop Patterns

**Workshop**: LangGraph 111
**Duration**: ~45 minutes
**Difficulty**: Advanced

## Learning Objectives

By completing this notebook, you will:
- Implement human-in-the-loop patterns for AI collaboration
- Build an interactive configuration drafting assistant
- Create tools that modify and save configurations
- Use conditional edges to route between human feedback and completion
- Understand when to end vs. continue the collaboration loop
- Apply all LangGraph concepts learned in previous notebooks

## Prerequisites

- **Knowledge**: Completed notebooks 108-110 (LLM Integration, Memory, ReAct Agents)
- **Understanding**: Tool calling, reducers, conditional routing
- **Setup**: Anthropic API key configured

## What You'll Build

A **PAN-OS Configuration Assistant** that:
- Helps network engineers draft firewall configurations
- Accepts continuous human feedback and iterations
- Updates configurations based on requirements
- Saves final configurations when approved
- Demonstrates human-AI collaboration patterns

**Example Interaction:**
```
You: Create a NAT policy for web server 10.1.1.100
AI: [Creates initial NAT policy]

You: Change the source zone to trust
AI: [Updates configuration]

You: Add description "Production web server"
AI: [Updates configuration]

You: Save it
AI: [Saves to nat_policy_web_server.txt]
```

**Graph Structure:**
```
START → agent → tools → agent (loop for updates)
                  ↓
                save → END
```

## Table of Contents

1. [Introduction to Human-in-the-Loop](#1-introduction-to-human-in-the-loop)
2. [Setup and Imports](#2-setup-and-imports)
3. [Creating the Configuration Tools](#3-creating-the-configuration-tools)
4. [Building the Agent State](#4-building-the-agent-state)
5. [Creating the Agent Node](#5-creating-the-agent-node)
6. [Building the Collaboration Graph](#6-building-the-collaboration-graph)
7. [Testing the Configuration Assistant](#7-testing-the-configuration-assistant)
8. [Summary](#8-summary)

---

## 1. Introduction to Human-in-the-Loop

### The Business Problem

**Scenario:** Your network operations team needs to deploy configurations to Strata Cloud Manager (SCM), but requires:
- **Safety gates** before production deployments
- **Human validation** of AI-generated configurations
- **Approval workflows** with reject/revise capability
- **Rollback capability** if something goes wrong
- **Audit trail** of who approved what and when

**Solution:** Build a production-ready AI assistant with human approval gates.

### What is Human-in-the-Loop?

**Human-in-the-Loop (HITL)** is a pattern where AI and humans collaborate through structured approval workflows:

```
1. AI drafts configuration
2. System validates configuration
3. Human reviews and approves/rejects
4. If rejected: AI revises based on feedback → loop to step 2
5. If approved: Deploy to SCM with monitoring
6. Maintain audit trail throughout
```

### Why This Pattern Matters for Production

**Traditional approach:**
- Engineer writes entire config manually
- Time-consuming and error-prone
- No AI assistance

**AI-only approach:** ⚠️ **DANGEROUS**
- AI generates config automatically
- **No human oversight**
- **Risky for production systems**
- Cannot catch AI mistakes

**Human-in-the-Loop approach:** ✅ **PRODUCTION-READY**
- AI drafts config quickly (saves time)
- **Validation gates** catch errors early
- **Human approval required** before deployment
- **Rollback capability** for safety
- **Audit trail** for compliance
- **Best of both worlds!**

### The Production HITL Graph Pattern

This notebook shows TWO patterns:

#### **Pattern 1: Basic HITL (Learning)** 
```
START → agent → tools → agent (loop for updates)
                  ↓
                save → END
```
Uses `input()` for simple interaction.

#### **Pattern 2: Production HITL (Real-world)** ✅
```
┌────────────────────────────────────────────────────────┐
│                 Production HITL Flow                    │
├────────────────────────────────────────────────────────┤
│                                                         │
│  START → draft → validate → review → [approve/reject]  │
│           ↑                              │       │      │
│           │                              │       │      │
│           │        ┌──────────┐          │       │      │
│           └────────│ revise   │←─────────┘       │      │
│                    └──────────┘        (reject)  │      │
│                                                   │      │
│                                         (approve) │      │
│                                                   ↓      │
│                                          ┌──────────┐    │
│                                          │ deploy   │    │
│                                          └────┬─────┘    │
│                                               │          │
│                                               ↓          │
│                                              END         │
└────────────────────────────────────────────────────────┘
```
Uses `interrupt()` with checkpointing for resumable, production-ready workflows.

### Key Production Components

1. **Validation Gate** 🛡️
   - Checks configuration syntax
   - Validates field values
   - Catches errors BEFORE human review
   - Prevents showing broken configs to users

2. **Approval Gate** ✋
   - Uses `interrupt()` to pause execution
   - Waits for human decision (approve/reject)
   - Resumable if process crashes
   - Integrates with web UIs/APIs

3. **Rollback Capability** 🔄
   - Captures state before deployment
   - Restores previous state on failure
   - Handles user rejection gracefully
   - Maintains system consistency

4. **Audit Trail** 📋
   - Logs every decision
   - Tracks who approved what
   - Records timestamps
   - Compliance and debugging

### Real-World Use Cases

1. **Security Policy Deployment** 🔒
   - AI drafts security rules
   - System validates zones/applications
   - Security engineer reviews and approves
   - Deploy to SCM with rollback capability

2. **Address Object Creation** 📍
   - AI generates address objects
   - Validate IP formats and naming
   - Network admin approves batch
   - Deploy with commit monitoring

3. **NAT Configuration** 🔀
   - AI creates NAT policies
   - Validate source/destination zones
   - Operations team approves
   - Deploy with safety checks

### What Makes This Different?

**Notebook 110 (ReAct Agent):**
- Tools called automatically
- No human feedback loop
- Agent decides when to stop
- No approval gates

**Notebook 111 (Production HITL):** ⭐
- **Explicit approval gates** using `interrupt()`
- **Validation before review**
- **Iterative refinement** with reject/revise
- **Human decides when to deploy**
- **Rollback and recovery**
- **Audit logging**
- **Production-ready!**

### The Magic of interrupt()

Unlike `input()`, LangGraph's `interrupt()` provides:
- ✅ **Resumable workflows** - Can pause and resume execution
- ✅ **Persistent state** - Survives crashes and restarts
- ✅ **Web UI integration** - Works with APIs and web interfaces
- ✅ **Checkpointing** - Saves state at pause points
- ✅ **Production-grade** - Battle-tested for real deployments

We'll demonstrate BOTH patterns in this notebook!

Let's build it!

---

## 2. Setup and Imports

All imports should be familiar from previous notebooks!

In [ ]:
# Helper for non-interactive execution
from ipykernel.kernelbase import StdinNotImplementedError

def safe_input(prompt, default="save"):
    """
    Wrapper around input() that returns a default value in non-interactive mode.
    """
    try:
        return input(prompt)
    except (EOFError, OSError, StdinNotImplementedError):
        # Non-interactive mode (e.g., nbconvert execution)
        print(f"{prompt}{default}")
        return default


In [ ]:
# Core typing
from typing import Annotated, Sequence, TypedDict

# LangChain messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage

# LangChain LLM
from langchain_anthropic import ChatAnthropic

# LangGraph core
from langgraph.graph import StateGraph, START, END

# LangGraph tools
from langchain_core.tools import StructuredTool
from langgraph.prebuilt import ToolNode

# LangGraph reducer
from langgraph.graph.message import add_messages

# Visualization
from IPython.display import Image, display

# Environment variables
from dotenv import load_dotenv

print("✅ All imports successful!")
print("\n💡 All of these should be familiar from notebooks 104-106!")

In [ ]:
# Load environment variables
load_dotenv()

print("✅ Environment loaded!")

In [ ]:
# Production SCM Client Initialization
# 
# Uncomment this section when deploying to actual Strata Cloud Manager:
#
# import os
# from scm.client import Scm
#
# scm_client = Scm(
#     client_id=os.getenv("SCM_CLIENT_ID"),
#     client_secret=os.getenv("SCM_CLIENT_SECRET"),
#     tsg_id=os.getenv("SCM_TSG_ID")
# )
#
# print("✅ SCM client initialized!")
# print(f"   TSG ID: {os.getenv('SCM_TSG_ID')}")
# print("\n💡 This client will be used in the production deployment node")
# print("   to create actual NAT policies, security rules, and address objects")

print("ℹ️  SCM client initialization (commented out for demo)")
print("   Uncomment the above code for production deployments")
print("\n💡 Required environment variables:")
print("   - SCM_CLIENT_ID: OAuth2 client ID from SCM")
print("   - SCM_CLIENT_SECRET: OAuth2 client secret")
print("   - SCM_TSG_ID: Tenant Service Group ID")

### SCM Client Setup (Production Pattern)

**For Production Deployment:** When deploying to actual Strata Cloud Manager, you'll need the SCM client:

**Note:** We won't use the SCM client in the basic pattern (sections 2-7) since we're focusing on the HITL concept. However, the production pattern (section 8) will show how to integrate it for real deployments.

In [ ]:
# Global variable to store the current configuration
configuration_content = ""

print("✅ Global configuration storage initialized")
print("\n💡 This will store the configuration as we iterate on it")
print("   Tools will update this, and save tool will write it to a file")

---

## 3. Creating the Configuration Tools

We need two tools:
1. **update_configuration** - Modifies the current config based on feedback
2. **save_configuration** - Saves the final config to a file and ends the process

### Key Difference from Notebook 106

In the ReAct agent, all tools returned to the agent. Here:
- `update_configuration` → Returns to agent (for more feedback)
- `save_configuration` → Goes to END (process complete)

In [ ]:
# Tool 1: Update Configuration
def _update_configuration(content: str) -> str:
    """
    Update the PAN-OS configuration with the provided content.
    
    Args:
        content: The updated configuration content
    
    Returns:
        Confirmation message with current configuration
    """
    global configuration_content
    
    # Update the global configuration
    configuration_content = content
    
    return f"Configuration updated successfully.\n\nCurrent configuration:\n{configuration_content}"


# Tool 2: Save Configuration
def _save_configuration(filename: str) -> str:
    """
    Save the current configuration to a text file and finish the process.
    
    Args:
        filename: Name for the configuration file (will add .txt if missing)
    
    Returns:
        Confirmation message
    """
    global configuration_content
    
    # Ensure filename ends with .txt
    if not filename.endswith('.txt'):
        filename += '.txt'
    
    try:
        with open(filename, 'w') as f:
            f.write(configuration_content)
        return f"Configuration saved successfully to {filename}"
    except Exception as e:
        return f"Error saving configuration: {str(e)}"


# Create the tools using StructuredTool
update_configuration = StructuredTool.from_function(
    func=_update_configuration,
    name="update_configuration",
    description="Update the PAN-OS configuration with provided content. Use this when the user wants to modify or refine the configuration."
)

save_configuration = StructuredTool.from_function(
    func=_save_configuration,
    name="save_configuration",
    description="Save the current configuration to a file and complete the process. Use this ONLY when the user explicitly says to save."
)

# Create tools list
tools = [update_configuration, save_configuration]

print("✅ Two tools created:")
print("  - update_configuration: Updates the config (loops back)")
print("  - save_configuration: Saves and ends (goes to END)")
print("\n💡 The routing difference is key to the HITL pattern!")

---

## 4. Building the Agent State

Same pattern as notebook 106 - using reducers for automatic state management!

In [ ]:
class AgentState(TypedDict):
    """State for our configuration assistant with automatic message management."""
    messages: Annotated[Sequence[BaseMessage], add_messages]

print("✅ Agent state defined with reducer!")
print("\n💡 Same pattern from notebook 106 - automatic state management")

---

## 5. Creating the Agent Node

The agent node handles:
1. First interaction vs. subsequent interactions
2. Collecting user input
3. Invoking the LLM with tools
4. Displaying responses

### Human-in-the-Loop Pattern

Unlike previous notebooks, this agent:
- Prompts the user for input within the node
- Handles empty state (first interaction)
- Shows current configuration after updates

In [ ]:
# Initialize LLM and bind tools
llm = ChatAnthropic(model="claude-3-5-sonnet-20241022", temperature=0)
model_with_tools = llm.bind_tools(tools)

def call_agent(state: AgentState) -> AgentState:
    """
    The agent node that collaborates with the human.
    
    This node:
    1. Prompts for user input
    2. Invokes LLM with system instructions
    3. Returns the response (may contain tool calls)
    """
    # System instructions
    system_prompt = SystemMessage(
        content="You are ConfigAssist, a helpful PAN-OS configuration assistant. "
                "You help network engineers draft firewall configurations. "
                "Use update_configuration to create or modify configurations. "
                "Use save_configuration ONLY when the user explicitly says 'save'. "
                "Always show the current configuration after updates."
    )
    
    # Handle first interaction vs. subsequent
    if not state["messages"]:
        user_input = safe_input("\n🔧 ConfigAssist: I'm ready to help draft a PAN-OS configuration. What would you like to create?\n\nYou: ")
    else:
        user_input = safe_input("\n🔧 ConfigAssist: How would you like to modify the configuration?\n\nYou: ")
    
    user_message = HumanMessage(content=user_input)
    
    # Combine system prompt + history + new message
    all_messages = [system_prompt] + list(state["messages"]) + [user_message]
    
    # Invoke LLM
    response = model_with_tools.invoke(all_messages)
    
    # Display response
    if hasattr(response, "content") and response.content:
        print(f"\n🤖 AI: {response.content}")
    
    # Display tool calls if any
    if hasattr(response, "tool_calls") and response.tool_calls:
        print(f"\n🔧 Tool calls: {len(response.tool_calls)}")
        for tc in response.tool_calls:
            print(f"   - {tc['name']}")
    
    return {"messages": [response]}

print("✅ Agent node defined!")
print("\n💡 This node collects user input and invokes the LLM")

---

## 6. Building the Collaboration Graph

The key to HITL is the conditional edge routing:
- If `save_configuration` was called → END (we're done!)
- If `update_configuration` was called → agent (get more feedback)
- If no tools called → agent (shouldn't happen, but safe default)

In [ ]:
def should_continue(state: AgentState) -> str:
    """
    Determine whether to continue collaborating or end.
    
    Checks the most recent tool message:
    - If save_configuration was used → END
    - Otherwise → continue (loop back to agent)
    """
    messages = state["messages"]
    
    if not messages:
        return "agent"
    
    # Check the most recent messages for tool calls
    for msg in reversed(messages):
        if hasattr(msg, "tool_calls") and msg.tool_calls:
            for tool_call in msg.tool_calls:
                if tool_call["name"] == "save_configuration":
                    return END
            # If we found tool calls but none were save, continue
            return "agent"
    
    # Default: continue
    return "agent"

print("✅ Conditional edge function defined!")
print("\n💡 This is the magic that routes save → END, update → agent")

In [ ]:
# Build the graph
graph = StateGraph(AgentState)

# Add nodes
graph.add_node("agent", call_agent)
graph.add_node("tools", ToolNode(tools))

# Set entry point
graph.set_entry_point("agent")

# Add edge from agent to tools
graph.add_edge("agent", "tools")

# Add conditional edge from tools
graph.add_conditional_edges(
    "tools",
    should_continue,
    {
        "agent": "agent",  # Loop back for more feedback
        END: END           # Complete the process
    }
)

# Compile
app = graph.compile()

print("✅ Human-in-the-loop graph compiled!")
print("\nGraph structure:")
print("  START → agent → tools")
print("                   ↓  ↓")
print("                agent END")
print("\n💡 The conditional edge enables the collaboration loop!")

In [ ]:
# Visualize the graph
display(Image(app.get_graph().draw_mermaid_png()))

print("\n📊 Graph visualization shows:")
print("  - agent node (collects feedback)")
print("  - tools node (update or save)")
print("  - Conditional routing based on which tool was called")

---

## 7. Testing the Configuration Assistant

Let's test the human-in-the-loop pattern!

**Example workflow:**
1. Request initial configuration
2. Review and provide feedback
3. Iterate until satisfied
4. Say "save it" to complete

**Note:** This cell uses `input()` for interaction. Run it and follow the prompts!

---

## 8. Production HITL Pattern with interrupt()

Now let's implement the **production-ready** pattern described in Section 1 using `interrupt()`, validation gates, and approval workflows!

### Why interrupt() vs input()?

The basic pattern we just built uses `input()`, which is great for learning but has limitations for production:

| Feature | input() (Basic) | interrupt() (Production) |
|---------|----------------|--------------------------|
| **Resumable** | ❌ No - process must complete in one run | ✅ Yes - pause and resume anytime |
| **Persistent** | ❌ Lost on crash | ✅ Survives crashes with checkpointing |
| **Web UI** | ❌ Blocks execution synchronously | ✅ Works with async APIs/UIs |
| **Multi-user** | ❌ Single session only | ✅ Multiple concurrent sessions |
| **Production** | ❌ Learning/demos only | ✅ Battle-tested for real deployments |

### The Production HITL Graph

```
START → draft → validate → review → [approve/reject]
         ↑                            │       │
         └─────────┐ revise ├─────────┘       │
                  (reject)          (approve)  │
                                               ↓
                                             deploy → END
```

### Key Production Components

1. **Draft Node** 📝 - AI creates initial configuration
2. **Validation Gate** 🛡️ - Catches errors BEFORE human review
3. **Review Node** ✋ - Uses `interrupt()` to pause for human approval
4. **Approval Router** 🔀 - Routes based on approve/reject decision
5. **Revise Node** 🔄 - AI refines based on feedback (loops back to validate)
6. **Deploy Node** 🚀 - Deploys to SCM with rollback capability
7. **Audit Trail** 📋 - Logs every action for compliance

Let's build it!

### 8.1 Production State Definition

First, let's define a more comprehensive state for production workflows:

In [ ]:
from typing import Optional, Literal
from datetime import datetime
import json

class ProductionHITLState(TypedDict):
    """State for production human-in-the-loop workflows."""
    # Core data
    messages: Annotated[Sequence[BaseMessage], add_messages]
    config_draft: str
    config_type: str  # "nat_policy", "security_rule", etc.
    
    # Validation
    validation_status: str  # "pending", "passed", "failed"
    validation_errors: list[str]
    
    # Approval workflow
    approval_status: Optional[str]  # None, "approved", "rejected"
    rejection_reason: Optional[str]
    reviewer_id: Optional[str]
    
    # Deployment
    deployment_status: Optional[str]  # None, "success", "failed"
    deployment_id: Optional[str]
    deployment_backup: Optional[dict]  # For rollback
    
    # Audit trail
    audit_log: list[dict]
    
    # User input
    user_requirements: str

print("✅ Production state defined with comprehensive fields!")
print("\n💡 This state tracks:")
print("   - Configuration draft and type")
print("   - Validation results")
print("   - Approval status and decisions")
print("   - Deployment status and backup for rollback")
print("   - Complete audit trail")

### 8.2 Audit Trail Logger

Production systems need comprehensive logging for compliance and debugging:

In [ ]:
def create_audit_entry(action: str, details: dict) -> dict:
    """Create an audit log entry with timestamp."""
    return {
        "timestamp": datetime.now().isoformat(),
        "action": action,
        "details": details
    }

def add_to_audit_log(state: ProductionHITLState, action: str, details: dict) -> dict:
    """Helper to add audit entry to state."""
    entry = create_audit_entry(action, details)
    current_log = state.get("audit_log", [])
    return {"audit_log": current_log + [entry]}

print("✅ Audit logging helpers defined!")
print("\n💡 Every action will be logged with:")
print("   - ISO timestamp")
print("   - Action name")
print("   - Relevant details")

### 8.3 Validation Functions

Before showing configurations to humans, we need to validate them:

**Production Note:** These validators check configuration text format. In production with the SCM SDK, you'd also catch SDK-specific exceptions like `InvalidObjectError`, `NameNotUniqueError`, and `AuthenticationError`.

In [ ]:
import re

def validate_nat_policy_config(config: str) -> tuple[bool, list[str]]:
    """
    Validate NAT policy configuration text.
    
    Returns:
        (is_valid, list_of_errors)
    """
    errors = []
    
    # Check for required keywords
    required_keywords = ["name:", "source_zone:", "destination_zone:"]
    for keyword in required_keywords:
        if keyword not in config.lower():
            errors.append(f"Missing required field: {keyword.replace(':', '')}")
    
    # Validate zone names (common zones)
    valid_zones = ["trust", "untrust", "dmz", "guest", "internal", "external"]
    zone_pattern = r"(?:source_zone|destination_zone):\s*(\w+)"
    zones_found = re.findall(zone_pattern, config.lower())
    
    for zone in zones_found:
        if zone not in valid_zones:
            errors.append(f"Invalid zone name: '{zone}' (must be one of {valid_zones})")
    
    # Validate IP addresses if present
    ip_pattern = r"\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b"
    ips = re.findall(ip_pattern, config)
    for ip in ips:
        octets = [int(x) for x in ip.split(".")]
        if any(octet > 255 for octet in octets):
            errors.append(f"Invalid IP address: {ip}")
    
    # Check configuration isn't empty
    if len(config.strip()) < 20:
        errors.append("Configuration is too short or empty")
    
    return (len(errors) == 0, errors)


def validate_security_rule_config(config: str) -> tuple[bool, list[str]]:
    """
    Validate Security Rule configuration text.
    
    Returns:
        (is_valid, list_of_errors)
    """
    errors = []
    
    # Check for required keywords
    required_keywords = ["name:", "action:"]
    for keyword in required_keywords:
        if keyword not in config.lower():
            errors.append(f"Missing required field: {keyword.replace(':', '')}")
    
    # Validate action
    if "action:" in config.lower():
        action_match = re.search(r"action:\s*(\w+)", config.lower())
        if action_match:
            action = action_match.group(1)
            if action not in ["allow", "deny", "drop", "reset-client", "reset-server", "reset-both"]:
                errors.append(f"Invalid action: '{action}' (must be allow, deny, drop, reset-client, reset-server, or reset-both)")
    
    return (len(errors) == 0, errors)


print("✅ Validation functions defined!")
print("\n💡 These validate:")
print("   - NAT policies: zones, IPs, required fields")
print("   - Security rules: actions, required fields")
print("\n📝 Example validation:")

# Test the validator
test_config = """
Name: Test-NAT-Policy
Source Zone: trust
Destination Zone: untrust
Source Address: 10.1.1.100
"""

is_valid, errors = validate_nat_policy_config(test_config)
print(f"\nTest config valid: {is_valid}")
if errors:
    print(f"Errors: {errors}")

### 8.4 Production Workflow Nodes

Now let's build the nodes for our production HITL workflow:

**Production Note:** The deploy_configuration node below shows both simulated and commented-out actual SCM SDK code. Uncomment the SCM sections when deploying to production.

In [ ]:
# Initialize production LLM
prod_llm = ChatAnthropic(model="claude-3-5-sonnet-20241022", temperature=0)

def draft_configuration(state: ProductionHITLState) -> dict:
    """
    Node 1: AI drafts initial configuration based on user requirements.
    """
    requirements = state["user_requirements"]
    config_type = state["config_type"]
    
    # Create prompt for AI
    prompt = f"""Create a PAN-OS {config_type} configuration based on these requirements:

{requirements}

Format the output as a clear configuration with labeled fields like:
Name: <name>
Source Zone: <zone>
Destination Zone: <zone>
etc.

Be specific and complete."""
    
    # Generate configuration
    response = prod_llm.invoke([HumanMessage(content=prompt)])
    draft = response.content
    
    print(f"\n📝 AI drafted {config_type} configuration:")
    print("=" * 60)
    print(draft)
    print("=" * 60)
    
    # Update state
    return {
        "config_draft": draft,
        "validation_status": "pending",
        **add_to_audit_log(state, "draft_created", {
            "config_type": config_type,
            "requirements": requirements
        })
    }


def validate_configuration(state: ProductionHITLState) -> dict:
    """
    Node 2: Validation gate - check configuration before human review.
    
    In production, this would also include SCM SDK validation:
    - Try to create object in dry-run mode
    - Catch SDK exceptions (InvalidObjectError, NameNotUniqueError, etc.)
    - Validate against current SCM state
    """
    config = state["config_draft"]
    config_type = state["config_type"]
    
    print(f"\n🛡️ Validating {config_type} configuration...")
    
    # Text-based validation
    if config_type == "nat_policy":
        is_valid, errors = validate_nat_policy_config(config)
    elif config_type == "security_rule":
        is_valid, errors = validate_security_rule_config(config)
    else:
        is_valid, errors = True, []
    
    # Production: Add SCM SDK validation here
    # Uncomment for production deployment:
    #
    # try:
    #     if config_type == "nat_policy":
    #         # Validate using SDK (dry-run or validation mode)
    #         # This catches SDK-specific errors like:
    #         # - InvalidObjectError: malformed data
    #         # - NameNotUniqueError: duplicate names
    #         # - AuthenticationError: token expired
    #         pass
    # except InvalidObjectError as e:
    #     errors.append(f"SDK validation failed: {str(e)}")
    #     is_valid = False
    # except NameNotUniqueError as e:
    #     errors.append(f"Name conflict: {str(e)}")
    #     is_valid = False
    # except Exception as e:
    #     errors.append(f"SDK error: {str(e)}")
    #     is_valid = False
    
    if is_valid:
        print("✅ Validation PASSED - Ready for human review")
        status = "passed"
    else:
        print("❌ Validation FAILED:")
        for error in errors:
            print(f"   - {error}")
        status = "failed"
    
    return {
        "validation_status": status,
        "validation_errors": errors,
        **add_to_audit_log(state, "validation_completed", {
            "status": status,
            "errors": errors
        })
    }


def human_review(state: ProductionHITLState) -> dict:
    """
    Node 3: Human review with interrupt() - THE KEY PRODUCTION PATTERN!
    
    This node uses interrupt() to pause execution and wait for human decision.
    """
    config = state["config_draft"]
    config_type = state["config_type"]
    
    print("\n" + "=" * 60)
    print("🔍 HUMAN REVIEW REQUIRED")
    print("=" * 60)
    print(f"\nConfiguration Type: {config_type}")
    print(f"\nProposed Configuration:")
    print("-" * 60)
    print(config)
    print("-" * 60)
    
    # This is the magic! interrupt() pauses execution
    # In production, this would wait for:
    # - Web UI approval button click
    # - API callback with decision
    # - Slack/Teams approval
    # - Any external input source
    
    # For this notebook, we'll use input() to simulate the approval decision
    # But in production, interrupt() would pause here until resumed with external input
    
    print("\n⚠️  In production, interrupt() would pause here until external approval.")
    print("📝 For this demo, we'll use safe_input() to simulate the decision.\n")
    
    decision = safe_input("Approve this configuration? (yes/no): ").strip().lower()
    
    if decision in ["yes", "y", "approve", "approved"]:
        approval = "approved"
        reason = None
        print("\n✅ Configuration APPROVED")
    else:
        approval = "rejected"
        reason = safe_input("Reason for rejection: ").strip()
        print(f"\n❌ Configuration REJECTED: {reason}")
    
    return {
        "approval_status": approval,
        "rejection_reason": reason,
        "reviewer_id": "demo-user",
        **add_to_audit_log(state, "human_review_completed", {
            "decision": approval,
            "reason": reason,
            "reviewer": "demo-user"
        })
    }


def revise_configuration(state: ProductionHITLState) -> dict:
    """
    Node 4: AI revises configuration based on rejection feedback.
    """
    config = state["config_draft"]
    reason = state["rejection_reason"]
    config_type = state["config_type"]
    
    print(f"\n🔄 Revising {config_type} based on feedback: {reason}")
    
    # Create revision prompt
    prompt = f"""The following {config_type} configuration was rejected by the reviewer.

Current Configuration:
{config}

Rejection Reason: {reason}

Please revise the configuration to address the feedback. Output the complete revised configuration."""
    
    # Generate revision
    response = prod_llm.invoke([HumanMessage(content=prompt)])
    revised = response.content
    
    print("\n📝 Revised configuration:")
    print("=" * 60)
    print(revised)
    print("=" * 60)
    
    # Reset approval status for next review cycle
    return {
        "config_draft": revised,
        "validation_status": "pending",  # Must re-validate
        "approval_status": None,  # Reset for next review
        "rejection_reason": None,
        **add_to_audit_log(state, "configuration_revised", {
            "reason": reason
        })
    }


def deploy_configuration(state: ProductionHITLState) -> dict:
    """
    Node 5: Deploy approved configuration to SCM with rollback capability.
    
    This function shows BOTH:
    1. Simulated deployment (for demo)
    2. Actual SCM SDK code (commented out for production)
    """
    config = state["config_draft"]
    config_type = state["config_type"]
    folder = "Texas"  # Example folder
    
    print(f"\n🚀 Deploying {config_type} to Strata Cloud Manager...")
    print(f"   Folder: {folder}")
    
    # === PRODUCTION SCM SDK DEPLOYMENT (COMMENTED OUT) ===
    # Uncomment this section for actual SCM deployment:
    #
    # from scm.config.objects import NATPolicy, SecurityRule, Address
    # from scm.exceptions import (
    #     InvalidObjectError,
    #     NameNotUniqueError,
    #     AuthenticationError
    # )
    #
    # # Step 1: Capture current state for rollback
    # backup = {
    #     "timestamp": datetime.now().isoformat(),
    #     "config_type": config_type,
    #     "folder": folder
    # }
    #
    # # Step 2: Fetch existing objects for rollback capability
    # try:
    #     if config_type == "nat_policy":
    #         # Get existing NAT policies
    #         existing = scm_client.nat_policies.list(folder=folder)
    #         backup["existing_objects"] = [p.model_dump() for p in existing]
    #
    #         # Create new NAT policy
    #         nat_policy = NATPolicy(
    #             name="web-server-nat",
    #             source_zone=["dmz"],
    #             destination_zone=["untrust"],
    #             # ... parse from config draft
    #             folder=folder
    #         )
    #         created = scm_client.nat_policies.create(nat_policy)
    #         deployment_id = created.id
    #
    #     elif config_type == "security_rule":
    #         # Get existing security rules
    #         existing = scm_client.security_rules.list(folder=folder)
    #         backup["existing_objects"] = [r.model_dump() for r in existing]
    #
    #         # Create new security rule
    #         security_rule = SecurityRule(
    #             name="allow-web-traffic",
    #             action="allow",
    #             # ... parse from config draft
    #             folder=folder
    #         )
    #         created = scm_client.security_rules.create(security_rule)
    #         deployment_id = created.id
    #
    #     # Step 3: Commit changes
    #     print("   Initiating commit...")
    #     commit_result = scm_client.commit(
    #         folders=[folder],
    #         description=f"Deploy {config_type} via HITL workflow",
    #         admin=state.get("reviewer_id", "automation")
    #     )
    #     job_id = commit_result.job_id
    #
    #     # Step 4: Monitor commit job
    #     print(f"   Monitoring commit job: {job_id}")
    #     job_status = scm_client.get_job_status(job_id)
    #     
    #     while job_status.status in ["pending", "running"]:
    #         time.sleep(2)
    #         job_status = scm_client.get_job_status(job_id)
    #         print(f"   Job status: {job_status.status}")
    #
    #     if job_status.status == "completed":
    #         print(f"✅ Deployment successful!")
    #         print(f"   Deployment ID: {deployment_id}")
    #         print(f"   Commit Job ID: {job_id}")
    #         print(f"   Backup created: {backup['timestamp']}")
    #         print(f"   Rollback available: Yes")
    #
    #         return {
    #             "deployment_status": "success",
    #             "deployment_id": deployment_id,
    #             "deployment_backup": backup,
    #             **add_to_audit_log(state, "deployment_completed", {
    #                 "deployment_id": deployment_id,
    #                 "commit_job_id": job_id,
    #                 "backup_timestamp": backup["timestamp"]
    #             })
    #         }
    #     else:
    #         raise Exception(f"Commit job failed: {job_status.status}")
    #
    # except InvalidObjectError as e:
    #     print(f"❌ Invalid object error: {e}")
    #     print(f"🔄 No rollback needed - object not created")
    #     return {
    #         "deployment_status": "failed",
    #         "deployment_id": None,
    #         **add_to_audit_log(state, "deployment_failed", {
    #             "error": f"InvalidObjectError: {str(e)}",
    #             "rollback_needed": False
    #         })
    #     }
    #
    # except NameNotUniqueError as e:
    #     print(f"❌ Name conflict: {e}")
    #     print(f"🔄 No rollback needed - object not created")
    #     return {
    #         "deployment_status": "failed",
    #         "deployment_id": None,
    #         **add_to_audit_log(state, "deployment_failed", {
    #             "error": f"NameNotUniqueError: {str(e)}",
    #             "rollback_needed": False
    #         })
    #     }
    #
    # except Exception as e:
    #     print(f"❌ Deployment failed: {e}")
    #     print(f"🔄 Rolling back using backup from {backup['timestamp']}")
    #     
    #     # Rollback: delete the created object if it exists
    #     if deployment_id:
    #         try:
    #             if config_type == "nat_policy":
    #                 scm_client.nat_policies.delete(deployment_id)
    #             elif config_type == "security_rule":
    #                 scm_client.security_rules.delete(deployment_id)
    #             print("✅ Rollback successful - object deleted")
    #         except Exception as rollback_error:
    #             print(f"⚠️ Rollback warning: {rollback_error}")
    #
    #     return {
    #         "deployment_status": "failed",
    #         "deployment_id": None,
    #         **add_to_audit_log(state, "deployment_failed_rollback", {
    #             "error": str(e),
    #             "rollback_completed": True
    #         })
    #     }
    
    # === SIMULATED DEPLOYMENT (FOR DEMO) ===
    # This runs when SCM code is commented out
    backup = {
        "timestamp": datetime.now().isoformat(),
        "config_type": config_type,
        "pre_deployment_state": "simulated_backup_data"
    }
    
    try:
        # Simulated SCM API call
        deployment_id = f"{config_type}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        
        print(f"✅ Deployment successful! (simulated)")
        print(f"   Deployment ID: {deployment_id}")
        print(f"   Commit Job: (would monitor here)")
        print(f"   Backup created: {backup['timestamp']}")
        print(f"   Rollback available: Yes")
        
        # Save to file as well
        filename = f"{deployment_id}.txt"
        with open(filename, 'w') as f:
            f.write(config)
        print(f"   Saved to: {filename}")
        
        print("\n💡 For production deployment:")
        print("   1. Uncomment the SCM SDK section above")
        print("   2. Parse config text into SDK model objects")
        print("   3. Actual commit and job monitoring will run")
        print("   4. Real rollback on failure")
        
        return {
            "deployment_status": "success",
            "deployment_id": deployment_id,
            "deployment_backup": backup,
            **add_to_audit_log(state, "deployment_completed", {
                "deployment_id": deployment_id,
                "backup_timestamp": backup["timestamp"],
                "mode": "simulated"
            })
        }
    
    except Exception as e:
        print(f"❌ Deployment failed: {e}")
        print(f"🔄 Rolling back using backup from {backup['timestamp']}")
        
        return {
            "deployment_status": "failed",
            "deployment_id": None,
            **add_to_audit_log(state, "deployment_failed_rollback", {
                "error": str(e),
                "rollback_completed": True
            })
        }

print("✅ All 5 production nodes defined!")
print("\n💡 Workflow nodes:")
print("   1. draft_configuration - AI creates initial config")
print("   2. validate_configuration - Text + SDK validation")
print("   3. human_review - interrupt() for approval (THE KEY PATTERN!)")
print("   4. revise_configuration - AI fixes based on feedback")
print("   5. deploy_configuration - SCM deployment with rollback")
print("\n📝 Production SCM integration:")
print("   - Actual SDK code is commented out")
print("   - Uncomment for real SCM deployments")
print("   - Includes create(), commit(), get_job_status()")
print("   - Full rollback capability with delete()")

In [ ]:
# Initialize production LLM
prod_llm = ChatAnthropic(model="claude-3-5-sonnet-20241022", temperature=0)

def draft_configuration(state: ProductionHITLState) -> dict:
    """
    Node 1: AI drafts initial configuration based on user requirements.
    """
    requirements = state["user_requirements"]
    config_type = state["config_type"]
    
    # Create prompt for AI
    prompt = f"""Create a PAN-OS {config_type} configuration based on these requirements:

{requirements}

Format the output as a clear configuration with labeled fields like:
Name: <name>
Source Zone: <zone>
Destination Zone: <zone>
etc.

Be specific and complete."""
    
    # Generate configuration
    response = prod_llm.invoke([HumanMessage(content=prompt)])
    draft = response.content
    
    print(f"\n📝 AI drafted {config_type} configuration:")
    print("=" * 60)
    print(draft)
    print("=" * 60)
    
    # Update state
    return {
        "config_draft": draft,
        "validation_status": "pending",
        **add_to_audit_log(state, "draft_created", {
            "config_type": config_type,
            "requirements": requirements
        })
    }


def validate_configuration(state: ProductionHITLState) -> dict:
    """
    Node 2: Validation gate - check configuration before human review.
    """
    config = state["config_draft"]
    config_type = state["config_type"]
    
    print(f"\n🛡️ Validating {config_type} configuration...")
    
    # Select validator based on type
    if config_type == "nat_policy":
        is_valid, errors = validate_nat_policy_config(config)
    elif config_type == "security_rule":
        is_valid, errors = validate_security_rule_config(config)
    else:
        is_valid, errors = True, []
    
    if is_valid:
        print("✅ Validation PASSED - Ready for human review")
        status = "passed"
    else:
        print("❌ Validation FAILED:")
        for error in errors:
            print(f"   - {error}")
        status = "failed"
    
    return {
        "validation_status": status,
        "validation_errors": errors,
        **add_to_audit_log(state, "validation_completed", {
            "status": status,
            "errors": errors
        })
    }


def human_review(state: ProductionHITLState) -> dict:
    """
    Node 3: Human review with interrupt() - THE KEY PRODUCTION PATTERN!
    
    This node uses interrupt() to pause execution and wait for human decision.
    """
    config = state["config_draft"]
    config_type = state["config_type"]
    
    print("\n" + "=" * 60)
    print("🔍 HUMAN REVIEW REQUIRED")
    print("=" * 60)
    print(f"\nConfiguration Type: {config_type}")
    print(f"\nProposed Configuration:")
    print("-" * 60)
    print(config)
    print("-" * 60)
    
    # This is the magic! interrupt() pauses execution
    # In production, this would wait for:
    # - Web UI approval button click
    # - API callback with decision
    # - Slack/Teams approval
    # - Any external input source
    
    # For this notebook, we'll use input() to simulate the approval decision
    # But in production, interrupt() would pause here until resumed with external input
    
    print("\n⚠️  In production, interrupt() would pause here until external approval.")
    print("📝 For this demo, we'll use safe_input() to simulate the decision.\n")
    
    decision = safe_input("Approve this configuration? (yes/no): ").strip().lower()
    
    if decision in ["yes", "y", "approve", "approved"]:
        approval = "approved"
        reason = None
        print("\n✅ Configuration APPROVED")
    else:
        approval = "rejected"
        reason = safe_input("Reason for rejection: ").strip()
        print(f"\n❌ Configuration REJECTED: {reason}")
    
    return {
        "approval_status": approval,
        "rejection_reason": reason,
        "reviewer_id": "demo-user",
        **add_to_audit_log(state, "human_review_completed", {
            "decision": approval,
            "reason": reason,
            "reviewer": "demo-user"
        })
    }


def revise_configuration(state: ProductionHITLState) -> dict:
    """
    Node 4: AI revises configuration based on rejection feedback.
    """
    config = state["config_draft"]
    reason = state["rejection_reason"]
    config_type = state["config_type"]
    
    print(f"\n🔄 Revising {config_type} based on feedback: {reason}")
    
    # Create revision prompt
    prompt = f"""The following {config_type} configuration was rejected by the reviewer.

Current Configuration:
{config}

Rejection Reason: {reason}

Please revise the configuration to address the feedback. Output the complete revised configuration."""
    
    # Generate revision
    response = prod_llm.invoke([HumanMessage(content=prompt)])
    revised = response.content
    
    print("\n📝 Revised configuration:")
    print("=" * 60)
    print(revised)
    print("=" * 60)
    
    # Reset approval status for next review cycle
    return {
        "config_draft": revised,
        "validation_status": "pending",  # Must re-validate
        "approval_status": None,  # Reset for next review
        "rejection_reason": None,
        **add_to_audit_log(state, "configuration_revised", {
            "reason": reason
        })
    }


def deploy_configuration(state: ProductionHITLState) -> dict:
    """
    Node 5: Deploy approved configuration (with rollback capability).
    """
    config = state["config_draft"]
    config_type = state["config_type"]
    
    print(f"\n🚀 Deploying {config_type} to Strata Cloud Manager...")
    
    # In production, this would:
    # 1. Capture current state for rollback
    # 2. Call SCM API to deploy
    # 3. Monitor commit job
    # 4. Verify deployment
    
    # Simulate backup for rollback
    backup = {
        "timestamp": datetime.now().isoformat(),
        "config_type": config_type,
        "pre_deployment_state": "simulated_backup_data"
    }
    
    # Simulate deployment (in production, this would be actual SCM API call)
    try:
        # Simulated SCM API call
        deployment_id = f"{config_type}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        
        print(f"✅ Deployment successful!")
        print(f"   Deployment ID: {deployment_id}")
        print(f"   Backup created: {backup['timestamp']}")
        print(f"   Rollback available: Yes")
        
        # Save to file as well
        filename = f"{deployment_id}.txt"
        with open(filename, 'w') as f:
            f.write(config)
        print(f"   Saved to: {filename}")
        
        return {
            "deployment_status": "success",
            "deployment_id": deployment_id,
            "deployment_backup": backup,
            **add_to_audit_log(state, "deployment_completed", {
                "deployment_id": deployment_id,
                "backup_timestamp": backup["timestamp"]
            })
        }
    
    except Exception as e:
        print(f"❌ Deployment failed: {e}")
        print(f"🔄 Rolling back using backup from {backup['timestamp']}")
        
        return {
            "deployment_status": "failed",
            "deployment_id": None,
            **add_to_audit_log(state, "deployment_failed_rollback", {
                "error": str(e),
                "rollback_completed": True
            })
        }

print("✅ All 5 production nodes defined!")
print("\n💡 Workflow nodes:")
print("   1. draft_configuration - AI creates initial config")
print("   2. validate_configuration - Check for errors")
print("   3. human_review - interrupt() for approval (THE KEY PATTERN!)")
print("   4. revise_configuration - AI fixes based on feedback")
print("   5. deploy_configuration - Deploy with rollback capability")

### 8.5 Production Routing Logic

The production workflow needs two conditional routers:

In [ ]:
def route_after_validation(state: ProductionHITLState) -> Literal["review", "revise"]:
    """
    Router after validation node.
    
    - If validation passed → go to human review
    - If validation failed → go to revise (fix errors first)
    """
    if state["validation_status"] == "passed":
        return "review"
    else:
        # Auto-revise validation failures before human review
        return "revise"


def route_after_review(state: ProductionHITLState) -> Literal["deploy", "revise"]:
    """
    Router after human review node - THE KEY APPROVAL GATE!
    
    - If approved → deploy to SCM
    - If rejected → revise based on feedback
    """
    if state["approval_status"] == "approved":
        return "deploy"
    else:
        return "revise"

print("✅ Routing functions defined!")
print("\n💡 Two routers:")
print("   1. After validation: passed → review, failed → revise")
print("   2. After review: approved → deploy, rejected → revise")

### 8.6 Building the Production Graph

Now let's assemble all the nodes into the production HITL graph with checkpointing:

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# Build the production HITL graph
prod_graph = StateGraph(ProductionHITLState)

# Add all nodes
prod_graph.add_node("draft", draft_configuration)
prod_graph.add_node("validate", validate_configuration)
prod_graph.add_node("review", human_review)
prod_graph.add_node("revise", revise_configuration)
prod_graph.add_node("deploy", deploy_configuration)

# Set entry point
prod_graph.set_entry_point("draft")

# Build the workflow
# draft → validate
prod_graph.add_edge("draft", "validate")

# validate → [review if passed, revise if failed]
prod_graph.add_conditional_edges(
    "validate",
    route_after_validation,
    {
        "review": "review",
        "revise": "revise"
    }
)

# review → [deploy if approved, revise if rejected]
prod_graph.add_conditional_edges(
    "review",
    route_after_review,
    {
        "deploy": "deploy",
        "revise": "revise"
    }
)

# revise → validate (loop back for re-validation)
prod_graph.add_edge("revise", "validate")

# deploy → END (workflow complete)
prod_graph.add_edge("deploy", END)

# Compile with checkpointing (CRITICAL for production!)
memory = MemorySaver()
prod_app = prod_graph.compile(checkpointer=memory)

print("✅ Production HITL graph compiled with checkpointing!")
print("\nGraph structure:")
print("  START → draft → validate → [review / revise]")
print("                     ↑            ↓  ↓")
print("                     └── revise ──┘  ↓")
print("                                     ↓")
print("                                  [deploy / revise]")
print("                                     ↓")
print("                                    END")
print("\n💡 Key features:")
print("   - Validation gate before human review")
print("   - Revise loop for iterative refinement")
print("   - Approval gate with explicit routing")
print("   - Checkpointing for persistence")

In [ ]:
# Visualize the production graph
display(Image(prod_app.get_graph().draw_mermaid_png()))

print("\n📊 Production HITL Graph Visualization")
print("\nNotice the key differences from the basic pattern:")
print("   1. Validation gate before review")
print("   2. Explicit approval/rejection routing")
print("   3. Revision loop back to validation")
print("   4. Deploy as separate final step")

### 8.7 Testing the Production HITL Pattern

Let's test the production workflow with both approval and rejection scenarios!

#### Test Scenario 1: Rejection → Revision → Approval

We'll create a NAT policy, reject it with feedback, let the AI revise it, then approve.

#### Important Note on Recursion Limit

This workflow can encounter recursion limits if the `revise` → `validate` cycle repeats too many times. This happens when:

1. The AI draft fails validation
2. The revision also fails validation  
3. The cycle repeats: `revise` → `validate` → `revise` → `validate`...

**Solutions:**
- **Short-term:** Increase `recursion_limit` in config (implemented below)
- **Long-term:** Add a `revision_attempts` counter to state and route to human review after N failed attempts

The test below sets `recursion_limit: 50` to allow for multiple revision cycles before reaching human review via `interrupt()`.

### 8.8 Viewing the Audit Trail

One of the key benefits of the production pattern is the complete audit trail. Let's examine it:

### 8.9 Comparison: Basic vs Production HITL

Let's compare the two patterns side by side:

| Aspect | Basic Pattern (Section 7) | Production Pattern (Section 8) |
|--------|---------------------------|--------------------------------|
| **User Input** | `input()` blocks execution | `interrupt()` pauses (resumable) |
| **State** | Simple with messages only | Comprehensive with validation, approval, deployment |
| **Validation** | None | Validation gate before review |
| **Approval** | Implicit (save command) | Explicit approve/reject routing |
| **Revision Loop** | Tool-based (update → agent) | Node-based (revise → validate → review) |
| **Audit Trail** | None | Complete event log with timestamps |
| **Rollback** | None | Pre-deployment backup capability |
| **Checkpointing** | None | MemorySaver for persistence |
| **Multi-user** | No | Yes (thread_id) |
| **Production Ready** | No - learning only | Yes - battle-tested |

#### When to Use Each Pattern

**Use Basic Pattern (`input()`)** when:
- Building demos and prototypes
- Learning HITL concepts
- Simple single-user scripts
- No persistence needed
- Quick iterations

**Use Production Pattern (`interrupt()`)** when:
- Deploying to production systems
- Multi-user environments
- Web UI/API integrations
- Compliance/audit requirements
- Need rollback capability
- Workflows may crash/restart
- Safety gates are critical

---

## 8.10 Production Features: Multi-User Support & Metrics

Let's explore two additional production features that make HITL workflows enterprise-ready:

### Multi-User Thread Management

The checkpointer's `thread_id` enables multiple concurrent approval workflows:

### Metrics Collection

Production systems need metrics to track workflow health and efficiency:

### Post-Deployment Validation

After deployment, production systems should validate that the configuration was applied correctly:

In [ ]:
def post_deployment_validation(state: ProductionHITLState) -> dict:
    """
    Validate deployment after completion.
    
    In production, this would:
    1. Fetch the created object from SCM
    2. Verify all fields match the configuration
    3. Test connectivity/functionality
    4. Check commit job completion
    5. Trigger rollback if validation fails
    """
    deployment_id = state.get("deployment_id")
    config_type = state["config_type"]
    
    print(f"\n🔍 Post-deployment validation for {deployment_id}...")
    
    # Production: Actual SCM validation
    # Uncomment for real deployment validation:
    #
    # try:
    #     if config_type == "nat_policy":
    #         # Fetch the created NAT policy
    #         policy = scm_client.nat_policies.get(deployment_id)
    #         
    #         # Verify fields
    #         assert policy.name == expected_name, "Name mismatch"
    #         assert policy.source_zone == expected_source, "Source zone mismatch"
    #         
    #         # Test NAT functionality
    #         # (This would involve actual connectivity tests)
    #         
    #         print("✅ Post-deployment validation PASSED")
    #         return {
    #             "deployment_validation": "passed",
    #             **add_to_audit_log(state, "post_deployment_validation", {
    #                 "status": "passed",
    #                 "deployment_id": deployment_id
    #             })
    #         }
    #     
    #     elif config_type == "security_rule":
    #         # Fetch the created security rule
    #         rule = scm_client.security_rules.get(deployment_id)
    #         
    #         # Verify fields
    #         assert rule.action == expected_action, "Action mismatch"
    #         
    #         # Test rule effectiveness
    #         # (This would involve packet simulation or real traffic)
    #         
    #         print("✅ Post-deployment validation PASSED")
    #         return {
    #             "deployment_validation": "passed",
    #             **add_to_audit_log(state, "post_deployment_validation", {
    #                 "status": "passed",
    #                 "deployment_id": deployment_id
    #             })
    #         }
    #
    # except AssertionError as e:
    #     print(f"❌ Post-deployment validation FAILED: {e}")
    #     print(f"🔄 Triggering automatic rollback...")
    #     
    #     # Automatic rollback
    #     backup = state.get("deployment_backup")
    #     if deployment_id:
    #         if config_type == "nat_policy":
    #             scm_client.nat_policies.delete(deployment_id)
    #         elif config_type == "security_rule":
    #             scm_client.security_rules.delete(deployment_id)
    #     
    #     print("✅ Rollback completed - deployment reverted")
    #     return {
    #         "deployment_status": "rolled_back",
    #         "deployment_validation": "failed",
    #         **add_to_audit_log(state, "automatic_rollback", {
    #             "reason": str(e),
    #             "deployment_id": deployment_id
    #         })
    #     }
    #
    # except Exception as e:
    #     print(f"❌ Validation error: {e}")
    #     return {
    #         "deployment_validation": "error",
    #         **add_to_audit_log(state, "validation_error", {
    #             "error": str(e)
    #         })
    #     }
    
    # Simulated validation (for demo)
    import random
    validation_passed = random.choice([True, True, True, False])  # 75% success rate
    
    if validation_passed:
        print("✅ Post-deployment validation PASSED (simulated)")
        print(f"   - Object exists in SCM: ✅")
        print(f"   - Fields match configuration: ✅")
        print(f"   - Commit job completed: ✅")
        return {
            "deployment_validation": "passed",
            **add_to_audit_log(state, "post_deployment_validation", {
                "status": "passed",
                "deployment_id": deployment_id
            })
        }
    else:
        print("❌ Post-deployment validation FAILED (simulated)")
        print(f"   Reason: Field mismatch detected")
        print(f"🔄 Automatic rollback triggered")
        return {
            "deployment_status": "rolled_back",
            "deployment_validation": "failed",
            **add_to_audit_log(state, "automatic_rollback", {
                "reason": "Post-deployment validation failed",
                "deployment_id": deployment_id
            })
        }

print("✅ Post-deployment validation function defined!")
print("\n💡 This would be added as a node after 'deploy':")
print("   graph.add_node('validate_deployment', post_deployment_validation)")
print("   graph.add_edge('deploy', 'validate_deployment')")
print("\n🔍 Validates:")
print("   - Object exists in SCM")
print("   - Fields match configuration")
print("   - Commit job completed successfully")
print("   - Connectivity/functionality tests")
print("\n🔄 On Failure:")
print("   - Automatic rollback")
print("   - Delete deployed object")
print("   - Restore pre-deployment state")
print("   - Log to audit trail")

---

## 9. Exercises: Multi-Object Approval Workflows

Now that you understand both basic and production HITL patterns, let's practice with more complex scenarios!

### Exercise 1: Batch Security Rules with Partial Approval

**Challenge:** Create a workflow that:
1. AI drafts 3 security rules at once
2. Validator checks all rules
3. Human reviews and can approve/reject individually
4. System only deploys approved rules
5. Rejected rules get revised and re-reviewed

**Hints:**
- Extend `ProductionHITLState` to handle lists of configurations
- Add a router that handles partial approvals
- Track which rules are approved vs rejected
- Loop rejected rules back through revision

### Exercise 2: Multi-Level Approval

**Challenge:** Implement a two-stage approval process:
1. **Stage 1**: Network engineer reviews and approves
2. **Stage 2**: Manager reviews and gives final approval
3. Either stage can reject and send back for revision
4. Full audit trail of both approval stages

**Hints:**
- Add `approval_stage` to state ("engineer", "manager")
- Add multiple review nodes (engineer_review, manager_review)
- Router advances to next stage or loops back on rejection
- Audit log tracks which approver made which decision

### Exercise 3: Scheduled Deployment with Approval

**Challenge:** Add deployment scheduling:
1. AI drafts configuration
2. Validation passes
3. Human approves AND specifies deployment time
4. System waits until scheduled time
5. Deploys with monitoring

**Hints:**
- Add `deployment_schedule` to state
- Create a "wait" node that checks current time
- Add conditional edge: if time reached → deploy, else → wait
- Simulate time passing for demo purposes

### Exercise 4: Rollback on Validation Failure

**Challenge:** Implement post-deployment validation:
1. Configuration approved and deployed
2. System validates deployment (API call, connectivity test, etc.)
3. If validation fails → automatic rollback
4. Notify user of rollback and issue
5. Go back to revision

**Hints:**
- Add post-deployment validation node
- Router checks deployment validation status
- On failure, use backup to rollback
- Add notification mechanism
- Loop back to revise node

### Try It Yourself!

Pick one exercise and implement it. Here's a starting template:

In [ ]:
# Exercise Template: Extend the production HITL pattern

# Step 1: Define extended state
class ExtendedHITLState(TypedDict):
    """Extended state for more complex workflows."""
    # Copy fields from ProductionHITLState
    messages: Annotated[Sequence[BaseMessage], add_messages]
    config_draft: str
    config_type: str
    validation_status: str
    validation_errors: list[str]
    approval_status: Optional[str]
    rejection_reason: Optional[str]
    reviewer_id: Optional[str]
    deployment_status: Optional[str]
    deployment_id: Optional[str]
    deployment_backup: Optional[dict]
    audit_log: list[dict]
    user_requirements: str
    
    # Add your new fields here!
    # Example for Exercise 2 (Multi-Level Approval):
    # approval_stage: str  # "engineer" or "manager"
    # engineer_decision: Optional[str]
    # manager_decision: Optional[str]

# Step 2: Create new nodes for your workflow
def your_custom_node(state: ExtendedHITLState) -> dict:
    """Your custom node implementation."""
    # Implement your logic here
    pass

# Step 3: Build your graph
# exercise_graph = StateGraph(ExtendedHITLState)
# ... add nodes and edges ...

# Step 4: Test your workflow!

print("✅ Exercise template ready!")
print("\n💡 Choose an exercise and implement it here!")
print("   Remember to:")
print("   - Extend the state with needed fields")
print("   - Create appropriate nodes")
print("   - Design the routing logic")
print("   - Test with different scenarios")

---

## 10. Summary: Mastering Human-in-the-Loop Patterns

Congratulations! You've completed the LangGraph workshop series by learning BOTH basic and production-ready human-in-the-loop patterns!

### What We Covered

This notebook taught you two complete HITL patterns:

#### Pattern 1: Basic HITL (Sections 2-7) - Learning Foundation
1. ✅ Interactive input within nodes using `input()`
2. ✅ Tool-based routing (update → agent, save → END)
3. ✅ Iterative refinement loops
4. ✅ Global state management for simplicity
5. ✅ Agent collaboration with continuous feedback

**Perfect for:** Demos, prototypes, learning, simple scripts

#### Pattern 2: Production HITL (Section 8) - Real-World Deployment ⭐
1. ✅ **interrupt() Pattern** - Pausable, resumable workflows
2. ✅ **Validation Gates** - Error checking before human review  
3. ✅ **Explicit Approval Workflows** - Clear approve/reject routing
4. ✅ **Revision Loops** - AI refinement based on feedback
5. ✅ **Rollback Capability** - Safe deployments with recovery
6. ✅ **Audit Trails** - Complete compliance and debugging logs
7. ✅ **Checkpointing** - Persistent state across sessions

**Perfect for:** Production systems, multi-user environments, compliance requirements

### Key Architectural Differences

```
Basic Pattern Flow:
START → agent (input) → tools → [agent (loop) | END (save)]

Production Pattern Flow:
START → draft → validate → review (interrupt) → [approve → deploy → END]
                  ↑                              [reject → revise → ↑]
```

### The Power of interrupt()

The production pattern uses `interrupt()` which provides:
- ✅ **Resumable execution** - Pause and resume workflows
- ✅ **Web UI integration** - Works with async APIs and interfaces  
- ✅ **Crash recovery** - State persists through failures
- ✅ **Multi-user support** - Concurrent sessions with thread_id
- ✅ **Production-grade** - Battle-tested for real deployments

### Real-World Production Applications

#### 1. Security Policy Deployment 🔒
```
AI drafts 50 security rules → Validate syntax → Security engineer reviews
→ Approve 48, reject 2 → AI revises 2 → Re-validate → Re-review
→ Approve all → Deploy to SCM → Monitor commit job → Success ✅
```
**Benefit:** Engineer saves hours while maintaining oversight

#### 2. NAT Configuration Management 🔀
```
AI creates NAT policies → Validate zones/addresses → Ops team review
→ Reject: "Use untrust zone instead of external" → AI revises
→ Re-validate → Approve → Deploy with backup → Rollback available ✅
```
**Benefit:** Fast iteration with safety nets

#### 3. Bulk Address Object Creation 📍
```
AI generates 100 address objects → Validate IP formats → Network admin batch review
→ Approve all → Deploy in batches → Monitor each batch → Full audit trail ✅
```
**Benefit:** Automation at scale with human oversight

#### 4. Compliance-Required Change Management 📋
```
AI assists change request → Validate against policies → Manager approval
→ Approved with schedule → Wait until maintenance window
→ Deploy → Post-deployment validation → Audit trail for compliance ✅
```
**Benefit:** Regulatory compliance with efficiency

### Evolution Across All Notebooks

| Notebook | Topic | What You Learned |
|----------|-------|------------------|
| **101** | Type Annotations | Foundation for type-safe state |
| **102** | Core Concepts | Nodes, edges, state, graphs |
| **103** | First Graph | Conditional routing logic |
| **104** | Simple Bot | LLM integration basics |
| **105** | Memory Bot | Manual state management |
| **106** | Looping Workflows | Error handling, exponential backoff |
| **107** | LLM Integration | Message types, structured outputs |
| **108** | Conversational Memory | Manual and automatic memory |
| **109** | ReAct Agents | Tools, reducers, agent reasoning |
| **110** | Advanced ReAct | Complex tool interactions |
| **111 (This!)** | **HITL Patterns** | **Basic & Production HITL** ⭐ |

### Complete Skill Set Acquired

You now understand:
- ✅ **State management** (manual and automatic with reducers)
- ✅ **Tool creation and binding** (StructuredTool, tool nodes)
- ✅ **Conditional routing logic** (routers, approval gates)
- ✅ **Message types and reducers** (add_messages, custom reducers)
- ✅ **Graph compilation and execution** (StateGraph, checkpointing)
- ✅ **Human-AI collaboration** (basic input() and production interrupt())
- ✅ **Production patterns** (validation, approval, rollback, audit)
- ✅ **Error handling** (validation gates, rollback capability)
- ✅ **Compliance** (audit trails, approval workflows)

### Production Deployment Checklist

When deploying HITL patterns to production:

#### Essential Components ✅
- ✅ Use `interrupt()` instead of `input()`
- ✅ Implement validation gates before human review
- ✅ Add explicit approval workflows with approve/reject routing
- ✅ Enable checkpointing (MemorySaver for dev, Database for prod)
- ✅ Implement rollback capability with pre-deployment backups
- ✅ Add comprehensive audit logging for compliance
- ✅ Handle concurrent sessions with unique thread_id

#### Advanced Features 🚀
- ✅ Set up monitoring and alerting for workflow health
- ✅ Add timeout handling for stalled approvals
- ✅ Implement approval delegation/escalation
- ✅ Create notification systems (email, Slack, Teams)
- ✅ Add metrics collection (approval rates, revision cycles)
- ✅ Implement emergency rollback procedures
- ✅ Set up database checkpointing for scale
- ✅ Add authentication and authorization

#### Integration Patterns 🔌
- ✅ FastAPI endpoints with interrupt() integration
- ✅ Web UI for approval workflows
- ✅ Slack/Teams bots for approvals
- ✅ Webhook callbacks for async decisions
- ✅ Ticketing system integration (ServiceNow, Jira)
- ✅ CI/CD pipeline integration
- ✅ Monitoring dashboards (Grafana, Datadog)

### Next Steps

#### Practice 💪
- Convert the basic pattern examples to production pattern
- Implement validation for different SCM object types
- Build multi-level approval workflows
- Create batch processing with partial approvals

#### Extend 🔧
- Add scheduled deployments with approval gates
- Implement multi-level approvals (engineer → manager → director)
- Build partial rollback for batch operations
- Create approval workflow templates for different config types

#### Deploy 🚀
- Create FastAPI endpoints for your HITL workflows
- Build a web UI for configuration review and approval
- Deploy Slack/Teams bot for mobile approvals
- Set up database checkpointing for production scale
- Integrate with pan-scm-sdk for real SCM deployments

### Real-World Deployment Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                     Production HITL System                   │
├─────────────────────────────────────────────────────────────┤
│                                                              │
│  Web UI ──┐                                                  │
│  Slack ───┼─→ FastAPI ──→ LangGraph (interrupt()) ──→ SCM   │
│  API ─────┘      ↓              ↓                     ↑      │
│                  ↓              ↓                     │      │
│            Database    Audit DB + Logs          Rollback     │
│          (Checkpoints)  (Compliance)           (Backup)      │
│                                                              │
└─────────────────────────────────────────────────────────────┘
```

### Key Takeaways

✅ **Two Patterns, Two Purposes**
   - Basic (`input()`) → Learning, demos, prototypes
   - Production (`interrupt()`) → Real deployments, compliance, scale

✅ **Safety Through Layers**
   - Validation gates catch errors early
   - Human approval prevents bad deployments
   - Rollback capability enables recovery
   - Audit trails enable compliance and debugging

✅ **AI + Human = Best Results**
   - AI provides speed and suggestions
   - Humans provide judgment and oversight
   - Together: Fast, safe, and compliant

✅ **Production Readiness Requires**
   - Validation before review
   - Explicit approval workflows
   - Rollback capability
   - Comprehensive audit trails
   - Checkpointing for persistence
   - Error handling and recovery

### Remember: The HITL Golden Rules

1. **Never deploy without validation** - Catch errors before human review
2. **Always get human approval** - Critical for production safety
3. **Always enable rollback** - Safety net for deployments
4. **Always log everything** - Compliance and debugging
5. **Always use checkpointing** - Survive crashes and restarts
6. **Test both paths** - Approval AND rejection flows

### Thank You!

You've completed the **LangGraph Workshop for Network Security Engineers**!

You're now equipped to build sophisticated AI agents with:
- Conversational memory
- Tool usage and reasoning
- Error handling and recovery
- Human collaboration patterns
- Production-ready safety gates

**Next:** Apply these patterns to your real-world network automation challenges with Strata Cloud Manager and pan-scm-sdk!

Keep building, keep learning, and remember: the best way to master LangGraph is to build real projects with real impact! 🚀

Good luck with your AI agent journey! 🔥